# Copy-on-Write vs Merge-on-Read ?

## Configuring Spark on EMR Notebook

In [1]:
%%configure
{
    "conf": {
      "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
      "spark.sql.catalog.glue": "org.apache.iceberg.spark.SparkCatalog",
        "spark.sql.catalog.glue.catalog-impl":"org.apache.iceberg.aws.glue.GlueCatalog",
        "spark.sql.catalog.glue.io-impl":"org.apache.iceberg.aws.s3.S3FileIO",
        "spark.sql.catalog.glue.warehouse":"s3://aws-blog-post-bucket/glue/warehouse"
    }
}

## Reading NYC Taxi trips [data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page) for creating Iceberg Tables

In [2]:
from pyspark.sql.functions import lit, col

# Reading NYC Yellow Taxi Data
data_bucket = "aws-blog-post-bucket" # Change here
raw_data_path = f"s3://{data_bucket}/raw_data/nyc_tlc"
yellow_sep_df = spark.read.format("parquet").load(f"{raw_data_path}/yellow/sep2023/")
yellow_oct_df = spark.read.format("parquet").load(f"{raw_data_path}/yellow/oct2023/")
# Creating month and year column
yellow_sep_df = yellow_sep_df.withColumn("month", lit(9)) \
        .withColumn("year", lit(2023))
yellow_oct_df = yellow_oct_df.withColumn("month", lit(10)) \
        .withColumn("year", lit(2023))
yellow_df = yellow_sep_df.unionByName(yellow_oct_df)
yellow_df.groupBy("VendorID","month").count().show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1706077363286_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-------+
|VendorID|month|  count|
+--------+-----+-------+
|       1|    9| 731968|
|       2|    9|2113902|
|       6|    9|    852|
|       2|   10|2617320|
|       1|   10| 904463|
|       6|   10|    502|
+--------+-----+-------+

In [4]:
yellow_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6369007

## Creating COW and MOR Table with same data

In [5]:
## Creating 2 tables with same data
cow_table = "glue.blogs_db.yellow_taxi_trips_cow"
mor_table = "glue.blogs_db.yellow_taxi_trips_mor"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# creating an Iceberg table in glue catalog within nyc_tlc database. default compression is taken as 'zstd'
yellow_df.writeTo(cow_table).partitionedBy("year", "month").using("iceberg") \
            .tableProperty("format-version", "2") \
            .tableProperty("write.parquet.compression-codec", "snappy") \
            .create()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
yellow_df.writeTo(mor_table).partitionedBy("year", "month").using("iceberg") \
            .tableProperty("format-version", "2") \
            .tableProperty("write.parquet.compression-codec", "snappy") \
            .create()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Pre Analysis of tables after tables are created

### COW Table Anaysis

In [9]:
spark.sql(f"select * from {cow_table}.snapshots").orderBy(col("committed_at").desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id|operation|manifest_list                                                                                                                         |summary                                                                                                                                                                                                                                                                              

In [10]:
spark.sql(f"select * from {cow_table}.manifests").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+---------------------------------------------------+
|content|path                                                                                                          |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries                                |
+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+-----------------

In [8]:
spark.sql(f"select content, file_format, file_path, record_count from {cow_table}.files").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------+------------+
|content|file_format|file_path                                                                                                                                   |record_count|
+-------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------+------------+
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_cow/data/year=2023/month=9/00000-32-f4235c17-4534-409a-95eb-e417dc4cb7dd-00001.parquet |1048576     |
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_cow/data/year=2023/month=9/00002-34-f4235c17-4534-409a-95eb-e417dc4cb7dd-00001.parquet |1048576     |
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_cow/data/year=2023/month=9/00003-35-f4235c17-4

### MOR Table Anaysis

In [11]:
spark.sql(f"select * from {mor_table}.snapshots").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id|operation|manifest_list                                                                                                                         |summary                                                                                                                                                                                                                                                                              

In [12]:
spark.sql(f"select * from {mor_table}.manifests").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+---------------------------------------------------+
|content|path                                                                                                          |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries                                |
+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+-----------------

In [13]:
spark.sql(f"select content, file_format, file_path, record_count from {mor_table}.files").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------+------------+
|content|file_format|file_path                                                                                                                                   |record_count|
+-------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------+------------+
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00001-41-131e5ffa-a9ce-4b67-96d7-f49cae976d4d-00001.parquet |1048576     |
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00004-44-131e5ffa-a9ce-4b67-96d7-f49cae976d4d-00001.parquet |1048576     |
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00006-46-131e5ffa-a

## Altering table to set COW and MOR table properties

In [14]:
spark.sql(f"""ALTER TABLE {cow_table} SET TBLPROPERTIES (
 'write.delete.mode'='copy-on-write',
 'write.update.mode'='copy-on-write',
 'write.merge.mode'='copy-on-write'
)""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [15]:
spark.sql(f"""ALTER TABLE {mor_table} SET TBLPROPERTIES (
 'write.delete.mode'='merge-on-read',
 'write.update.mode'='merge-on-read',
 'write.merge.mode'='merge-on-read'
)""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

## Performing delete operation

In [16]:
# from cow table
spark.sql(f"DELETE from {cow_table} where VendorId=6") # took 11.38secs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [17]:
# from mor table
spark.sql(f"DELETE from {mor_table} where VendorId=6") # took 2.32secs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

### Analyzing COW Table for rewritten data files

In [18]:
spark.sql(f"select * from {cow_table}.snapshots").orderBy(col("committed_at").desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                         |summary                                                                                                                                                                      

In [26]:
latest_snapshot_id = 1999244325536185772
spark.sql(f"select * from {cow_table}.manifests where added_snapshot_id = {latest_snapshot_id}").drop("partition_summaries").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+
|content|path                                                                                                          |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|
+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+
|0      |s3://aws-blog-post-bucket/blogs

Things to look for in above output:
- `added_data_files_count` and `delete_data_files_count` show that files are rewritten.
- `deleted_data_files_count` shows that there are no Delete Files has been added.

In [20]:
## Analyzing tables after delete operation
spark.sql(f"select content, file_format, file_path, record_count from {cow_table}.files").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------+------------+
|content|file_format|file_path                                                                                                                                   |record_count|
+-------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------+------------+
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_cow/data/year=2023/month=10/00000-67-2c32246d-243a-4a86-bed8-114b66df5a7c-00001.parquet|376055      |
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_cow/data/year=2023/month=9/00001-68-2c32246d-243a-4a86-bed8-114b66df5a7c-00001.parquet |748718      |
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_cow/data/year=2023/month=9/00000-32-f4235c17-4

In `files` metadata table, `content` column value as:
- `0` represents a Data File.
- `1` represents a Positional Delete File.
- `2` represents a Equality Delete File.

`content` column in above output shows as `0`. That shows there are only data files present in the table when the delete operation happened with COW approach.

## Analyzing MOR table

In [21]:
spark.sql(f"select * from {mor_table}.snapshots").orderBy(col("committed_at").desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                         |summary                                                                                                                                                                                                                  

In [22]:
latest_snapshot_id = 799231277590704241
spark.sql(f"select * from {mor_table}.manifests where added_snapshot_id = {latest_snapshot_id}").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+---------------------------------------------------+
|content|path                                                                                                          |length|partition_spec_id|added_snapshot_id |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries                                |
+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+------------------+----------------------+-------------------------+------------------------+--------------------

Only Delete File is added in case of DELETE operation.

`added_delete_files_count` = `2` can be seen in above table.

In [23]:
## Analyzing MOR table
spark.sql(f"select content, file_format, file_path, record_count from {mor_table}.files").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|content|file_format|file_path                                                                                                                                           |record_count|
+-------+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00001-41-131e5ffa-a9ce-4b67-96d7-f49cae976d4d-00001.parquet         |1048576     |
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00004-44-131e5ffa-a9ce-4b67-96d7-f49cae976d4d-00001.parquet         |1048576     |
|0      |PARQUET    |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/da

`content` = `1` in above output shows the added Positional Delete Files from DELETE Operation.

In [24]:
## Checking contents in delete file
delete_file_path = "s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00000-71-902451fb-4e65-4817-b1ae-ea3a96f8eec8-00001-deletes.parquet"
delete_file_df = spark.read.parquet(delete_file_path)
delete_file_df.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------------------------------------------------------------------------+------+
|file_path                                                                                                                                  |pos   |
+-------------------------------------------------------------------------------------------------------------------------------------------+------+
|s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00006-46-131e5ffa-a9ce-4b67-96d7-f49cae976d4d-00001.parquet|609427|
|s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00006-46-131e5ffa-a9ce-4b67-96d7-f49cae976d4d-00001.parquet|610008|
|s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00006-46-131e5ffa-a9ce-4b67-96d7-f49cae976d4d-00001.parquet|610045|
|s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00006-46-131e5ffa-a9ce-4b

Positional Delete Files contains the `file_path` and the exact position of the records deleted/updated from that file.

## Performing update operation

### On MOR Table

In [25]:
# Performing an update operation
spark.sql(f"update {mor_table} set fare_amount = 0 where VendorID=2 and fare_amount < 0") # took 23.42 secs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

### Analyzing metadata table after Update operation

In [28]:
# Getting the recent snapshot_id for looking into manifest metadata table.
spark.sql(f"select * from {mor_table}.snapshots").orderBy(col("committed_at").desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                         |summary                                                                                                                                                              

In [29]:
latest_snapshot_id = 2049488298809185761
spark.sql(f"select * from {mor_table}.manifests").drop("partition_summaries").filter(col("added_snapshot_id") == latest_snapshot_id).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+
|content|path                                                                                                          |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|
+-------+--------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+
|0      |s3://aws-blog-post-bucket/blogs

`content` in `manifest` metadata table:
- `0` manifest file tracking the data files.
- `1` manifest file tracking the delete files.

In output above, look at the `added_data_files_count` and `added_delete_files_count` column values, this shows 2 new data files are added which contains the records with updated value and 2 delete files are added that contains the file_path and position of the updated record.

In [30]:
# Looking into files metadata table after UPDATE operation
spark.sql(f"select content, file_path, file_format, partition,record_count from {mor_table}.files").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+------------+
|content|file_path                                                                                                                                           |file_format|partition |record_count|
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+------------+
|0      |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=10/00001-88-630c6981-9422-40c3-acbf-2efeccf058e7-00001.parquet        |PARQUET    |{2023, 10}|37099       |
|0      |s3://aws-blog-post-bucket/blogs_db/yellow_taxi_trips_mor/data/year=2023/month=9/00002-89-630c6981-9422-40c3-acbf-2efeccf058e7-00001.parquet         |PARQUET    |{2023, 9} |29562       |
|0      |s3://aws-blog-po